In [22]:
# Download Amazon 2018 toy & games review and metadata
%cd ../Data/
%run download_amazon_data.py
%cd ../alex_fuzzy_matching/

/home/alex/Documents/erdos/summer-2025-safeify/Data
File already exists.
File already exists.
/home/alex/Documents/erdos/summer-2025-safeify/alex_fuzzy_matching


In [11]:
import pandas as pd

In [12]:
# Import first 100,000 entries of amazon metadata into a pandas dataframe.
# Obviously we'd want to do this on the full dataset, but my computer struggled importing the full dataset due to RAM limitations
meta_chunks=pd.read_json("../Data/amazon_meta.json",lines=True,chunksize=100_000)
metadata=next(meta_chunks)

In [25]:
# Import CPSC IncidentReports
reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')

# Filter to reports in the Toys and Children category. (There's also a baby category.)
reports=reports.loc[reports['Product Category']=='Toys & Children']

# Filter to reports of items manufactured before October 2018,
# since items manufuctered after this data won't appear in the amazon dataset
reports['Date Manufactured']=pd.to_datetime(reports['Date Manufactured'])
report=reports[reports['Date Manufactured']<='Oct 2018']

/tmp/ipykernel_58444/612986380.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  reports=pd.read_csv("../Data/IncidentReports.csv",encoding='cp1252')


In [14]:
# Features we can use for fuzzy matching.

print(metadata.columns)
print(reports.columns)

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')
Index(['Report No.', 'Report Date',
       'Sent to Manufacturer / Importer / Private Labeler', 'Publication Date',
       'Category of Submitter', 'Product Description', 'Product Category',
       'Product Sub Category', 'Product Type', 'Product Code',
       'Manufacturer / Importer / Private Labeler Name', 'Brand',
       'Model Name or Number', 'Serial Number', 'UPC', 'Date Manufactured',
       'Manufacturer Date Code', 'Retailer', 'Retailer State', 'Purchase Date',
       'Purchase Date Is Estimate', 'Incident Description', 'City', 'State',
       'ZIP', 'Location', '(Primary) Victim Severity',
       '(Primary) Victim's Sex', 'My Relation To The (Primary) Victim',
       '(Primary) Victim's Age (years)', 'Submitter Has Product',
 

In [15]:
import rapidfuzz as rf


In [16]:

def best_match(query,choices):
    """Function returning best match of "query" in "choices"
    Returns:
        tuple: A tuple consisting of the following elemnts:
            - The best match choice in choices
            - A normalized "edit distance"
            - The index of the best match.
    """
    return rf.process.extractOne(query,choices,scorer=rf.fuzz.ratio)


In [17]:
# Restricting to N sample reports to try and match them
N=50
reports_sample=reports.iloc[:N].copy()

In [18]:
reports_sample['best_brand_match_data']=reports_sample['Brand'].apply(lambda x: best_match(x,metadata['brand']))


In [19]:
reports_sample['Best Brand Match']=reports_sample['best_brand_match_data'].apply(lambda x : x[0] if x!=None else None)
reports_sample['Brand Match Score']=reports_sample['best_brand_match_data'].apply(lambda x : x[1] if x!=None else None)
reports_sample[['Brand','Best Brand Match','Brand Match Score']]

,Brand,Best Brand Match,Brand Match Score
5,Glow & Save,Soft & Safe,63.636364
18,Green Elephant Home and Toys,Great Wall Toys,55.813953
29,Anbalulu,Abali,61.538462
35,Walmart Way to Celebrate,Wild Planet Water Babies,48.979592
47,Bunnypal,Buengna,62.500000
48,HGY Toys,GC Toys,80.000000
74,Big Bang,Big Blast,70.588235
83,Brand: Zugar Land 4.0 4.0 out of 5 stars (19) ...,Friendship Fashion and Fun 45th Anniversary Fi...,38.834951
87,Guava Family Inc,Mitzvah Family,60.000000
139,Way To Celebrate,Candy Crate,59.259259


In [20]:
reports_sample['best_description_match_data']=reports_sample['Model Name or Number'].apply(lambda x: rf.process.extractOne(x,metadata['description'],scorer=rf.fuzz.ratio))


In [21]:
reports_sample['Best Desc. Match']=reports_sample['best_description_match_data'].apply(lambda x : x[0] if x!=None else None)
reports_sample['Desc. Match Score']=reports_sample['best_description_match_data'].apply(lambda x : x[1] if x!=None else None)
reports_sample[['Model Name or Number','Best Desc. Match','Desc. Match Score']]

,Model Name or Number,Best Desc. Match,Desc. Match Score
5,Sleeping Otter,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",0.000000
18,Pastel/Macaron colored,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",0.000000
29,AN-100,[N],28.571429
35,"Whale- EGT65209B, Dino- EGT65209C",[0],5.882353
47,Same,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",0.000000
48,NaN,None,NaN
74,NaN,None,NaN
83,Unknown,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",0.000000
87,Roam,"[Three Dr. Suess' Puzzles: Green Eggs and Ham,...",0.000000
139,Whale-EGT65209B,[0],12.500000
